In [ ]:
import ndef

# Define the NFC Capability Container (CC bytes)
magic_comment_readwrite = "0XE1, 0X40, 0X40, 0X00"  # Read/Write enabled
magic_comment_only_read = "0XE1, 0X40, 0X40, 0X01"   # Read-Only mode

def generate_nfc_c_table(records, read_only=False):
    """
    Generate a C-style array for an NFC Type 2 Tag storing any NDEF record(s).

    Args:
        records (list): A list of NDEF records (e.g., ndef.UriRecord, ndef.TextRecord).
        read_only (bool): Whether the tag should be read-only.

    Returns:
        str: C initialization table as a formatted string.
    """
    # Select the correct CC bytes
    magic_comment = magic_comment_only_read if read_only else magic_comment_readwrite

    # Encode all NDEF records
    ndef_payload = b''.join(ndef.message_encoder(records))

    # Compute the length of the NDEF message
    ndef_length = len(ndef_payload)

    # Construct the final NDEF message with Type 2 NFC header
    ndef_message = bytes([0x03, ndef_length]) + ndef_payload + b'\xFE'  # Add NDEF start + end marker

    # Convert to hexadecimal representation
    hex_message = ndef_message.hex().upper()
    
    # notify length 
    print("size is : " + str(4 +len(hex_message)/2))
    hex_split = [f"0X{hex_message[i:i+2]}" for i in range(0, len(hex_message), 2)]

    # Format as C array
    c_table = f"{{{magic_comment}, " + ', '.join(hex_split) + "};"
    
    return c_table

# Example: Generate C-style NFC table for multiple records (multiple are allowed)
records = [
    ndef.TextRecord("I was run    1 times")
    # ndef.UriRecord("youtube.com") # link (or location)
]
#note : Apple cant read Geo data and texts


print(generate_nfc_c_table(records, read_only=False))

size is : 34.0
{0XE1, 0X40, 0X40, 0X00, 0X03, 0X1B, 0XD1, 0X01, 0X17, 0X54, 0X02, 0X65, 0X6E, 0X49, 0X20, 0X77, 0X61, 0X73, 0X20, 0X72, 0X75, 0X6E, 0X20, 0X20, 0X20, 0X20, 0X31, 0X20, 0X74, 0X69, 0X6D, 0X65, 0X73, 0XFE};
